## 제목을 입력합니다

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from pykrx import stock
from sklearn.model_selection import train_test_split

### csv 불러오기 실습

DataFrame : 2차원 데이터 -> table

In [2]:
stock_info = pd.read_csv("stock.csv")
stock_info['num'] = range(0, len(stock_info))
stock_label = stock_info[['isin','num']]
stock_label

,isin,num
0,095570,0
1,006840,1
2,027410,2
3,282330,3
4,138930,4
...,...,...
948,005010,948
949,000540,949
950,000547,950
951,000545,951


In [3]:
ohlcv = pd.read_csv('ohlcv최종.csv', dtype={"isin":str})
ohlcv['isin'] = ohlcv['isin'].replace(stock_label.set_index('isin')['num'])
ohlcv

,isin,s_date,open,high,low,close,volume,amount,updown
0,0,2015-08-21,30900,36500,28000,35500,1994022,64841739950,14.89
1,0,2015-08-24,33500,34700,31200,32400,392831,12926256800,-8.73
2,0,2015-08-25,33050,33650,30950,31100,242730,7808534250,-4.01
3,0,2015-08-26,31300,38800,31300,35450,703721,25144972800,13.99
4,0,2015-08-27,36200,40300,34800,37850,499913,18764860300,6.77
...,...,...,...,...,...,...,...,...,...
2212948,952,2023-06-28,2090,2115,2055,2115,2042994,4239077890,-0.47
2212949,952,2023-06-29,2135,2135,2055,2055,1388206,2915012720,-2.84
2212950,952,2023-06-30,2050,2120,2030,2100,1594500,3339019670,2.19
2212951,952,2023-07-03,2140,2400,2120,2170,21631514,49189201040,3.33


In [4]:
date = pd.DataFrame(ohlcv['s_date'].unique())
date_label = date.rename({0:'s_date'}, axis = 1)
date_label = date_label.sort_values('s_date')
date_label['num'] = range(0, len(date))
date_label = date_label[['s_date','num']]
date_label

,s_date,num
1936,2013-01-02,0
1937,2013-01-03,1
1938,2013-01-04,2
1939,2013-01-07,3
1940,2013-01-08,4
...,...,...
1931,2023-06-28,2581
1932,2023-06-29,2582
1933,2023-06-30,2583
1934,2023-07-03,2584


In [5]:
grouped_close_list = pd.DataFrame([])
grouped_updown_list = pd.DataFrame([])
grouped_2derivative_list = pd.DataFrame([])

for isin, group in ohlcv.groupby('isin'):
    group['2derivative'] = group['updown'] - group['updown'].shift(1)
    group = group.fillna(0)
    tmp = group[['s_date','close']].set_index('s_date')
    tmp = tmp.rename(columns={'close': isin})
    grouped_close_list = grouped_close_list.join(pd.DataFrame(tmp), how = "outer")
    tmp = group[['s_date','updown']].set_index('s_date')
    tmp = tmp.rename(columns={'updown': isin})
    grouped_updown_list = grouped_updown_list.join(pd.DataFrame(tmp), how = "outer")
    tmp = group[['s_date','2derivative']].set_index('s_date')
    tmp = tmp.rename(columns={'2derivative': isin})
    grouped_2derivative_list = grouped_2derivative_list.join(pd.DataFrame(tmp), how = "outer")

In [6]:
grouped_updown_list = grouped_updown_list.reset_index().drop("s_date",axis=1).fillna(0).T
grouped_2derivative_list = grouped_2derivative_list.reset_index().drop("s_date",axis=1).fillna(0).T
grouped_close_list = grouped_close_list.reset_index().drop("s_date",axis=1).fillna(0).T

In [7]:
grouped_updown_list

,0,1,2,3,4,5,6,7,8,9,...,2576,2577,2578,2579,2580,2581,2582,2583,2584,2585
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.11,-0.99,-0.78,-0.22,1.46,0.33,-1.54,-0.67,0.90,-0.45
1,0.23,3.37,3.04,0.42,-0.42,-1.05,0.21,-2.13,1.52,-3.64,...,-2.90,3.14,-1.73,1.86,-1.67,0.31,0.10,-1.39,1.09,-0.41
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-1.13,0.00,-1.27,-0.39,0.39,-0.77,-1.30,0.79,0.65,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-1.66,-0.87,-1.37,2.34,-0.92,0.38,-2.19,-2.29,0.57,-3.24
4,1.89,2.23,1.09,0.72,0.71,-3.55,-0.37,1.48,-0.36,0.00,...,-0.29,0.73,-1.15,-0.73,1.47,0.72,-2.59,2.21,1.44,-0.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,1.21,2.19,-1.36,0.20,0.59,0.78,-0.97,-0.79,-1.58,-0.80,...,-2.23,-0.65,-2.45,1.85,1.81,-0.49,-1.46,1.49,2.44,0.00
949,-0.61,1.02,-1.31,0.10,0.10,0.31,-0.51,0.00,0.00,0.20,...,-0.47,-1.11,-1.76,0.82,0.16,0.97,-0.64,1.61,0.00,-0.63
950,-3.16,3.26,0.18,-0.35,1.23,0.69,-1.38,0.87,1.39,0.85,...,29.96,29.92,-17.19,-14.29,-2.22,1.36,6.28,-15.08,0.00,0.00
951,1.29,3.32,-2.47,0.00,-0.51,0.25,-1.78,1.03,0.00,-0.77,...,4.23,-0.17,-3.90,-1.06,-0.18,-0.18,-0.72,0.36,0.36,0.00


In [8]:
grouped_2derivative_list

,0,1,2,3,4,5,6,7,8,9,...,2576,2577,2578,2579,2580,2581,2582,2583,2584,2585
0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,1.95,-1.10,0.21,0.56,1.68,-1.13,-1.87,0.87,1.57,-1.35
1,0.0,3.14,-0.33,-2.62,-0.84,-0.63,1.26,-2.34,3.65,-5.16,...,-2.29,6.04,-4.87,3.59,-3.53,1.98,-0.21,-1.49,2.48,-1.50
2,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-1.00,1.13,-1.27,0.88,0.78,-1.16,-0.53,2.09,-0.14,-0.65
3,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-2.36,0.79,-0.50,3.71,-3.26,1.30,-2.57,-0.10,2.86,-3.81
4,0.0,0.34,-1.14,-0.37,-0.01,-4.26,3.18,1.85,-1.84,0.36,...,-0.29,1.02,-1.88,0.42,2.20,-0.75,-3.31,4.80,-0.77,-2.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,0.0,0.98,-3.55,1.56,0.39,0.19,-1.75,0.18,-0.79,0.78,...,-2.23,1.58,-1.80,4.30,-0.04,-2.30,-0.97,2.95,0.95,-2.44
949,0.0,1.63,-2.33,1.41,0.00,0.21,-0.82,0.51,0.00,0.20,...,-0.47,-0.64,-0.65,2.58,-0.66,0.81,-1.61,2.25,-1.61,-0.63
950,0.0,6.42,-3.08,-0.53,1.58,-0.54,-2.07,2.25,0.52,-0.54,...,49.03,-0.04,-47.11,2.90,12.07,3.58,4.92,-21.36,15.08,0.00
951,0.0,2.03,-5.79,2.47,-0.51,0.76,-2.03,2.81,-1.03,-0.77,...,4.41,-4.40,-3.73,2.84,0.88,0.00,-0.54,1.08,0.00,-0.36


In [9]:
grouped_close_list

,0,1,2,3,4,5,6,7,8,9,...,2576,2577,2578,2579,2580,2581,2582,2583,2584,2585
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4550.0,4505.0,4470.0,4460.0,4525.0,4540.0,4470.0,4440.0,4480.0,4460.0
1,21565.0,22292.0,22971.0,23068.0,22971.0,22728.0,22777.0,22292.0,22632.0,21808.0,...,19110.0,19710.0,19370.0,19730.0,19400.0,19460.0,19480.0,19210.0,19420.0,19340.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3925.0,3925.0,3875.0,3860.0,3875.0,3845.0,3795.0,3825.0,3850.0,3850.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,183900.0,182300.0,179800.0,184000.0,182300.0,183000.0,179000.0,174900.0,175900.0,170200.0
4,13044.0,13335.0,13480.0,13577.0,13674.0,13189.0,13141.0,13335.0,13286.0,13286.0,...,6890.0,6940.0,6860.0,6810.0,6910.0,6960.0,6780.0,6930.0,7030.0,6980.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,25100.0,25650.0,25300.0,25350.0,25500.0,25700.0,25450.0,25250.0,24850.0,24650.0,...,6150.0,6110.0,5960.0,6070.0,6180.0,6150.0,6060.0,6150.0,6300.0,6300.0
949,4915.0,4965.0,4900.0,4905.0,4910.0,4925.0,4900.0,4900.0,4900.0,4910.0,...,3155.0,3120.0,3065.0,3090.0,3095.0,3125.0,3105.0,3155.0,3155.0,3135.0
950,5520.0,5700.0,5710.0,5690.0,5760.0,5800.0,5720.0,5770.0,5850.0,5900.0,...,9760.0,12680.0,10500.0,9000.0,8800.0,8920.0,9480.0,8050.0,8050.0,8050.0
951,1960.0,2025.0,1975.0,1975.0,1965.0,1970.0,1935.0,1955.0,1955.0,1940.0,...,5910.0,5900.0,5670.0,5610.0,5600.0,5590.0,5550.0,5570.0,5590.0,5590.0


In [10]:
y = grouped_close_list.pop(2585)
X = grouped_close_list
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [12]:
model.score(X_test,y_test)

0.9970913877554729

In [13]:
model.predict(X_test)

array([2.96528670e+03, 6.06606568e+03, 1.05300327e+05, 1.69247370e+04,
       1.74553817e+03, 1.20802578e+04, 6.29087069e+03, 2.08416182e+03,
       4.53353664e+03, 1.68415565e+03, 6.57808069e+03, 2.29219041e+05,
       3.58005022e+04, 8.09571566e+03, 1.22855048e+04, 2.08073543e+03,
       7.00664680e+03, 9.07451489e+03, 2.88210541e+03, 7.23731637e+03,
       1.26412876e+03, 1.10913687e+02, 1.70115639e+05, 3.98677173e+03,
       1.92999993e+04, 2.21492240e+04, 3.56308730e+04, 1.34674053e+04,
       8.39111704e+03, 9.48771918e+04, 8.38939276e+02, 6.52186159e+04,
       2.61778350e+03, 1.69668881e+04, 2.86833836e+04, 2.69660856e+04,
       1.73768153e+03, 5.04138305e+03, 1.63896312e+03, 1.00791684e+04,
       3.54743564e+04, 2.79451774e+03, 7.94465925e+03, 1.11295720e+03,
       3.04185690e+04, 6.16784518e+02, 5.79396868e+04, 6.19310559e+03,
       4.41486057e+03, 1.01014262e+04, 1.06678981e+05, 6.52306261e+04,
       2.61221285e+03, 6.04247298e+03, 1.01023388e+03, 4.59909057e+03,
      

In [14]:
y_test

199      2880.0
420      5580.0
694    109700.0
750     16300.0
507      1516.0
         ...   
33       4900.0
31       5620.0
711      3775.0
299      2165.0
616     22850.0
Name: 2585, Length: 191, dtype: float64

In [15]:
isin = stock_label['isin']
isin

0      095570
1      006840
2      027410
3      282330
4      138930
        ...  
948    005010
949    000540
950    000547
951    000545
952    003280
Name: isin, Length: 953, dtype: object

In [16]:
date_label[date_label['s_date']==max(date_label['s_date'])]

,s_date,num
1935,2023-07-04,2585


In [18]:
df = stock.get_market_ticker_name("005930")
df

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
df = stock.get_market_ohlcv("20230704", "20230707", "005930")
df = df.reset_index()
tmp_map = max(date_label['num'])
df['날짜'] = df['날짜'].astype(str)
for s_date in df['날짜']:
    if s_date not in date_label['s_date'].values:
        new_row = pd.DataFrame({'s_date': [s_date], 'num': [tmp_map + 1]})
        date_label = pd.concat([date_label, new_row], ignore_index=True)
        tmp_map += 1
df['날짜'] = df['날짜'].replace(date_label.set_index('s_date')['num'])
df

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
date_label

,s_date,num
0,2013-01-02,0
1,2013-01-03,1
2,2013-01-04,2
3,2013-01-07,3
4,2013-01-08,4
...,...,...
2584,2023-07-03,2584
2585,2023-07-04,2585
2586,2023-07-05,2586
2587,2023-07-06,2587


In [ ]:
new_data = pd.DataFrame([])
date_set = ["20230703","20230704","20230705","20230706","20230707"]

for date in date_set:
    df = stock.get_market_ohlcv(date)
    df = df.reset_index()
    df["s_date"] = date[:4] + "-" + date[4:6] + "-" + date[6:]
    df = df.dropna(axis=0, how='any')
    new_data = pd.concat([new_data,df])
new_data = new_data.rename({"티커":"isin", "시가" : "open", "고가" : "high", "저가" : "low", "종가" : "close", "거래량" : "volume", "거래대금" : "amount", "등락률" : "updown"}, axis=1)
new_data = new_data[['isin', 's_date', 'open', 'high', 'low', 'close', 'volume', 'amount', 'updown']]
new_data = new_data.sort_values(['isin', 's_date'])
new_data['2derivative'] = new_data['updown'] - new_data['updown'].shift(1)
new_data = new_data[new_data["s_date"] != "2023-07-03"]
new_data = new_data.reset_index()
new_data.drop(['index'], axis=1, inplace=True)
new_data['isin'] = new_data['isin'].replace(stock_label.set_index('isin')['num'])
new_data['s_date'] = new_data['s_date'].replace(date_label.set_index('s_date')['num'])
new_data

,isin,s_date,open,high,low,close,volume,amount,updown,2derivative
0,340,2585,10300,10440,10260,10390,83550,865107990,0.48,0.96
1,340,2586,10390,10440,10090,10370,154517,1582256610,-0.19,-0.67
2,340,2587,10390,10390,10190,10300,92202,946743290,-0.68,-0.49
3,340,2588,10300,10300,9990,10100,123131,1243830270,-1.94,-1.26
4,84,2585,602,602,577,584,147115,86238821,-1.85,-2.36
...,...,...,...,...,...,...,...,...,...,...
3807,590,2588,2330,2450,2320,2420,1158393,2782227720,2.54,5.82
3808,796,2585,9300,9530,9240,9320,29216,272040120,-0.64,0.10
3809,796,2586,9230,9710,9200,9390,62709,589109300,0.75,1.39
3810,796,2587,9350,9370,9030,9050,47677,435120980,-3.62,-4.37


In [ ]:
new_close_list = pd.DataFrame([])
new_updown_list = pd.DataFrame([])
new_2derivative_list = pd.DataFrame([])

for isin, group in new_data.groupby('isin'):
    tmp = group[['s_date','close']].set_index('s_date')
    tmp = tmp.rename(columns={'close': isin})
    new_close_list = new_close_list.join(pd.DataFrame(tmp), how = "outer")
    tmp = group[['s_date','updown']].set_index('s_date')
    tmp = tmp.rename(columns={'updown': isin})
    new_updown_list = new_updown_list.join(pd.DataFrame(tmp), how = "outer")
    tmp = group[['s_date','2derivative']].set_index('s_date')
    tmp = tmp.rename(columns={'2derivative': isin})
    new_2derivative_list = new_2derivative_list.join(pd.DataFrame(tmp), how = "outer")

In [ ]:
new_close_list = new_close_list.fillna(0).T
new_updown_list = new_updown_list.fillna(0).T
new_2derivative_list = new_2derivative_list.fillna(0).T

In [ ]:
new_close_list

s_date,2585,2586,2587,2588
0,4460,4390,4275,4305
1,19340,19320,19080,19080
2,3850,3800,3715,3660
3,170200,167800,166600,166800
4,6980,6860,6730,6670
...,...,...,...,...
948,6300,6340,6160,6020
949,3135,3080,3070,3060
950,8050,8050,8050,9960
951,5590,5670,5530,5410


In [ ]:
new_updown_list

s_date,2585,2586,2587,2588
0,-0.45,-1.57,-2.62,0.70
1,-0.41,-0.10,-1.24,0.00
2,0.00,-1.30,-2.24,-1.48
3,-3.24,-1.41,-0.72,0.12
4,-0.71,-1.72,-1.90,-0.89
...,...,...,...,...
948,0.00,0.63,-2.84,-2.27
949,-0.63,-1.75,-0.32,-0.33
950,0.00,0.00,0.00,23.73
951,0.00,1.43,-2.47,-2.17


In [ ]:
new_2derivative_list

s_date,2585,2586,2587,2588
0,-1.35,-1.12,-1.05,3.32
1,-1.50,0.31,-1.14,1.24
2,-0.65,-1.30,-0.94,0.76
3,-3.81,1.83,0.69,0.84
4,-2.15,-1.01,-0.18,1.01
...,...,...,...,...
948,-2.44,0.63,-3.47,0.57
949,-0.63,-1.12,1.43,-0.01
950,0.00,0.00,0.00,23.73
951,-0.36,1.43,-3.90,0.30


In [ ]:
new_data = pd.concat([X, new_close_list], axis =1)
new_data
y_a = new_data.pop(2588)
x_a = new_data
y_b = new_data.pop(2587)
x_b = new_data
y_c = new_data.pop(2586)
x_c = new_data
y_d = new_data.pop(2585)
x_d = new_data

In [ ]:
model.score(x_a,y_a)

0.9971425653225064

In [ ]:
model.score(x_b,y_b)

0.9977121220656016

In [ ]:
model.score(x_c,y_c)

0.9993067817750125

In [ ]:
model.score(x_d,y_d)

0.9997167260628006

In [ ]:
a = pd.concat([pd.DataFrame(model.predict(x_a)), y_a],axis=1)
b = pd.concat([pd.DataFrame(model.predict(x_b)), y_b],axis=1)
c = pd.concat([pd.DataFrame(model.predict(x_c)), y_c],axis=1)
d = pd.concat([pd.DataFrame(model.predict(x_d)), y_d],axis=1)
a['error'] = round((abs((a[2588] - a[0]) / a[2588]) * 100), 2)
b['error'] = round((abs((b[2587] - b[0]) / b[2587]) * 100), 2)
c['error'] = round((abs((c[2586] - c[0]) / c[2586]) * 100), 2)
d['error'] = round((abs((d[2585] - d[0]) / d[2585]) * 100), 2)

In [ ]:
a

,0,2588,error
0,4460.000000,4305,3.60
1,19340.000000,19080,1.36
2,3850.000000,3660,5.19
3,170200.000000,166800,2.04
4,6980.000000,6670,4.65
...,...,...,...
948,5731.889083,6020,4.79
949,3135.000000,3060,2.45
950,8050.000000,9960,19.18
951,5590.000000,5410,3.33


In [ ]:
b

,0,2587,error
0,4460.000000,4275,4.33
1,19340.000000,19080,1.36
2,3850.000000,3715,3.63
3,170200.000000,166600,2.16
4,6980.000000,6730,3.71
...,...,...,...
948,5731.889083,6160,6.95
949,3135.000000,3070,2.12
950,8050.000000,8050,0.00
951,5590.000000,5530,1.08


In [ ]:
c

,0,2586,error
0,4460.000000,4390,1.59
1,19340.000000,19320,0.10
2,3850.000000,3800,1.32
3,170200.000000,167800,1.43
4,6980.000000,6860,1.75
...,...,...,...
948,5731.889083,6340,9.59
949,3135.000000,3080,1.79
950,8050.000000,8050,0.00
951,5590.000000,5670,1.41


In [ ]:
d

,0,2585,error
0,4460.000000,4460,0.00
1,19340.000000,19340,0.00
2,3850.000000,3850,0.00
3,170200.000000,170200,0.00
4,6980.000000,6980,0.00
...,...,...,...
948,5731.889083,6300,9.02
949,3135.000000,3135,0.00
950,8050.000000,8050,0.00
951,5590.000000,5590,0.00
